In [27]:
import tensorflow as tf
TF_LITE_MODEL_FILE_NAME = 'action.tflite'
interpreter = tf.lite.Interpreter(TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [28]:
input_details

[{'name': 'serving_default_lstm_input:0',
  'index': 0,
  'shape': array([  1,  50, 132]),
  'shape_signature': array([ -1,  50, 132]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [29]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 49,
  'shape': array([1, 3]),
  'shape_signature': array([-1,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [30]:
import numpy as np
import os
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

actions = np.array(['open' , 'put' , 'stand' ])
label_map = {label:num for num , label in enumerate(actions)}

In [4]:
no_sequences = 20
sequences_length = 50
DATA_PATH = os.path.join('mediapipe_data')


sequences , labels = [] , []
for action in actions :
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequences_length):
            res = np.load(os.path.join(DATA_PATH , action , str(sequence) , '{}.npy'.format(frame_num))) 
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [31]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [32]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

In [38]:

interpreter.resize_tensor_input(input_details[0]['index'], (60, 50, 132))
interpreter.resize_tensor_input(output_details[0]['index'], (60, 3))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [39]:
input_details

[{'name': 'serving_default_lstm_input:0',
  'index': 0,
  'shape': array([ 60,  50, 132]),
  'shape_signature': array([ -1,  50, 132]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [40]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 49,
  'shape': array([60,  3]),
  'shape_signature': array([-1,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [41]:
X = np.array(X , dtype = np.float32)
print(X.dtype)

float32


In [42]:
interpreter.set_tensor(input_details[0]['index'] , X)
interpreter.invoke()

In [43]:
np.argmax(interpreter.get_tensor(output_details[0]['index']) ,axis = 1)

array([1, 1, 2, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 1, 2, 2, 0, 0, 1, 2, 1,
       2, 2, 0, 2, 1, 0, 0, 1, 0, 2, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 1, 0,
       0, 2, 1, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 2, 2, 2], dtype=int64)

In [45]:
np.argmax(y , axis =1)

array([1, 1, 2, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 1, 2, 2, 0, 0, 1, 2, 1,
       2, 2, 0, 2, 1, 0, 0, 1, 0, 2, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 1, 0,
       0, 2, 1, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 2, 2, 2], dtype=int64)

In [46]:
from sklearn.metrics import accuracy_score

In [47]:
accuracy_score(np.argmax(interpreter.get_tensor(output_details[0]['index']) ,axis = 1),
np.argmax(y , axis =1)
               )

1.0

that is really good yay